In [1]:
from dateutil.relativedelta import *
from dateutil.easter import *
from dateutil.rrule import *
from dateutil.parser import *
from datetime import *
from sklearn.preprocessing import LabelEncoder

import conf
from UI.LOG import *
import cv2
from aml.train_pipeline import *  
from aml.train_pipeline import *
import pandas as pd

import os
import torchvision
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor,FasterRCNN_ResNet50_FPN_Weights
from torchinfo import summary

# from torchvision.models.detection import ssd300_vgg16,SSD300_VGG16_Weights
from torchvision.models.detection.ssd import SSDClassificationHead,SSD300_VGG16_Weights,det_utils
from torchvision.models.detection import ssd300_vgg16

import torch

import aml.model_using as model_using
import aml.support_func as support_funcS
import aml.time_mesuarment as time_mesuarment
import sys

import aml.managers as managers
import aml.img_processing as img_processing
import random
import numpy as np
import pprint
from torchinfo import summary
from aml.img_processing import *

from PIL import Image
import aml.models as models
import matplotlib.pyplot as plt
from pprint import pprint as Print
from PIL import Image
import warnings
from torchvision.utils import draw_bounding_boxes  
from torchvision.io.image import read_image
from torchvision.transforms.functional import to_pil_image
from torchvision.ops import nms 
from torchmetrics.detection.mean_ap import MeanAveragePrecision as mAP
from matplotlib.transforms import Affine2D
import mpl_toolkits.axisartist.floating_axes as floating_axes
from IPython.display import IFrame, display, HTML
from sklearn.model_selection import train_test_split
from catboost import CatBoostClassifier
from data_manip import label_encode_feautures_
import catboost
from io import StringIO 
import sys
from sklearn.metrics import roc_auc_score
from catboost import utils
from sklearn import metrics
import xgboost as xgb
from xgboost import XGBClassifier
from tqdm import tqdm
import lightgbm as lgb
from xgboost import plot_tree
tqdm.pandas()
# %matplotlib inline

def simple_plot(x,y,title=''):
    fig, ax = plt.subplots()
    fig.set_size_inches(16,9)
    ax.plot(x,y)
    ax.set_title(title)
    ax.grid()

    return fig,ax


def simple_plot_y1y2(x,y1,y2,label1='',label2='',title=''):
    fig, ax = plt.subplots()
    fig.set_size_inches(16,9)
    ax.plot(x,y1,label=label1)
    ax.plot(x,y2,label=label2)
    ax.set_title(title)
    ax.grid(which='both')
    ax.legend()
    return fig,ax

def simple_plot_y1y2y3y4(x,y1,y2,y3,y4,label1='',label2='',label3='',label4='',title=''):
    fig, ax = plt.subplots()
    fig.set_size_inches(16,9)
    ax.plot(x,y1,label=label1)
    ax.plot(x,y2,label=label2)
    ax.plot(x,y3,label=label3)
    ax.plot(x,y4,label=label4)
    ax.set_title(title)
    ax.grid(which='both')

    ax.legend()
    return fig,ax


def Gini(fpr,tpr):
    return 2*metrics.auc(fpr,tpr)-1
def plot_gxb_train_results(results):
    fig, ax = plt.subplots()
    fig.set_size_inches(16,9)
    ax.plot(results['train-auc-mean'],label= 'train-auc-mean')
    ax.plot(results['test-auc-mean'],label= 'test-auc-mean')
    ax.legend()
    return fig,ax

    

In [2]:
X = pd.read_csv(conf.X_train_dataset,index_col=False)
Y = pd.read_csv(conf.Y_train_dataset,index_col=False)

In [3]:
print(X.shape)
print([el for el in X])

(1199861, 277)
['acc_now_delinq', 'annual_inc', 'chargeoff_within_12_mths', 'collections_12_mths_ex_med', 'delinq_2yrs', 'dti', 'earliest_cr_line', 'funded_amnt', 'inq_last_12m', 'installment', 'int_rate', 'issue_d', 'loan_amnt', 'mort_acc', 'mths_since_last_delinq', 'mths_since_recent_bc_dlq', 'mths_since_recent_inq', 'num_accts_ever_120_pd', 'num_actv_bc_tl', 'num_rev_accts', 'num_sats', 'num_tl_120dpd_2m', 'num_tl_30dpd', 'num_tl_90g_dpd_24m', 'num_tl_op_past_12m', 'open_acc', 'open_il_24m', 'open_rv_24m', 'percent_bc_gt_75', 'pub_rec', 'pub_rec_bankruptcies', 'revol_util', 'tax_liens', 'total_acc', 'zip_code', 'fico_range_mid', 'emp_length_and_purpose_1', 'emp_length_and_purpose_2', 'emp_length_and_purpose_3', 'emp_length_and_purpose_4', 'emp_length_and_purpose_5', 'emp_length_and_purpose_6', 'emp_length_and_purpose_7', 'emp_length_and_purpose_8', 'emp_length_and_purpose_9', 'emp_length_and_purpose_10', 'emp_length_and_purpose_11', 'emp_length_and_purpose_12', 'emp_length_and_purpo

In [4]:
X_train, X_validation, y_train, y_validation = train_test_split(X, Y, train_size=0.8, random_state=1234)
features_names = [el for el in X]
cat_features = np.setdiff1d(label_encode_feautures_,np.setdiff1d(label_encode_feautures_,features_names))
print(cat_features)

['zip_code']


In [5]:
# dtrain_ = xgb.DMatrix(X_train, y_train, enable_categorical=True)
# dval_ = xgb.DMatrix(X_validation, y_validation, enable_categorical=True)
# all_train_ = xgb.DMatrix(X,Y,enable_categorical=True)

In [6]:
# def gen_random(mid_point,low_mul,high_mul,dtype,N_):
#     a_ = mid_point*low_mul
#     b_ = mid_point*high_mul
#     values = np.random.uniform(low=a_,high=b_,size = N_)
#     if dtype == 'int':
#         return np.array(values,dtype=np.int64)
#     if dtype == 'float':
#         return values

# def make_random_grid_search():
#     mid = {
#         'eta': 0.05,
#         'subsample': 0.2,
#         'max_depth': 6,
#         'learning_rate': 0.05,
#         'n_estimators': 2000,
#         'gamma':  0.1,
#         'subsample': 0.5,
#         'colsample_bytree':0.5, 
#         'colsample_bylevel':0.5, 
#         'colsample_bynode':0.5,
#         'lambda': 1.5,
#         'alpha': 0.1,
#         'scale_pos_weight': np.sum(np.where(Y==0))/np.sum(np.where(Y==1)),
#         'min_child_weight': 5,
#         'max_delta_step': 5,
#     }

#     N = 3
#     grid = pd.DataFrame({
#         'eta':                  gen_random(mid['eta'], 0.5, 2.0,'float',N),
#         'subsample':            gen_random(mid['subsample'], 0.5, 2.0,'float',N),
#         'max_depth':            gen_random(mid['max_depth'], 0.5, 2.0,'float',N),
#         'alpha'                 gen_random(mid['alpha'], 0.5, 2.0,'float',N): ,
#         'learning_rate':        gen_random(mid['learning_rate'], 0.5, 2.0,'float',N),
#         'n_estimators':         gen_random(mid['n_estimators'], 0.5, 2.0,'float',N),
#         'colsample_bytree':     gen_random(mid['colsample_bytree'], 0.5, 2.0,'float',N),
#         'gamma':                gen_random(mid['gamma'], 0.5, 2.0,'float',N): , 
#         'subsample'             gen_random(mid['subsample'], 0.5, 2.0,'float',N): ,
#         'colsample_bytree'      gen_random(mid['colsample_bytree'], 0.5, 2.0,'float',N): ,
#         'min_child_weight':     gen_random(mid['min_child_weight'], 0.5, 2.0,'float',N),
#         'max_delta_step':       gen_random(mid['max_delta_step'], 0.5, 2.0,'float',N),
#     })
#     return grid

# def fit(x):
#     params = {
#               'objective':'binary:logistic',
#               'eval_metric':'auc',
#               'eta':x[0],
#               'subsample':x[1],
#               }
#     xgb_cv = xgb.cv(dtrain=all_train_, params=params, nfold=5, metrics = 'logloss',seed=42,num_boost_round=10,early_stopping_rounds=5)
#     return xgb_cv[-1:].values[0]
# grid = make_random_grid_search()
# grid[['train-logloss-mean','train-logloss-std','test-logloss-mean','test-logloss-std']] = grid.progress_apply(fit, axis=1, result_type='expand')
# grid



In [7]:
eval_metric = 'auc'
# eval_metric = 'logloss'

params = {
    'eta': 0.01,
    # 'subsample': 0.0,
    'max_depth': 8,
    # 'learning_rate': 0.05,
    'n_estimators': 4000,
    'gamma':  0.2,
    'colsample_bytree':0.5, 
    'colsample_bylevel':0.5, 
    'colsample_bynode':0.5,
    'lambda': 1.5,
    'alpha': 0.1,
    'scale_pos_weight': np.sum(np.where(Y==0))/np.sum(np.where(Y==1)),
    'min_child_weight': 5,
    'max_delta_step': 5,
    'objective':'binary:logistic',
    'eval_metric': eval_metric, 
    'tree_method':'gpu_hist',
    # 'tree_method':'hist',
    'n_jobs':8,
    'gpu_id':0,
    'seed':0,
    'subsample': 0.9,
    # 'sampling_method': 'gradient_based',
    # 'enable_categorical': True,
    # 'max_cat_to_onehot': 1500
}    

model = XGBClassifier(**params)

In [8]:
model.fit(X,Y, eval_set=[(X_validation, y_validation),(X_train,y_train)],verbose=True)
# try:
    # model.fit(X,Y,verbose=True)
# except:
    # print('smth went wrong')

[0]	validation_0-auc:0.60512	validation_1-auc:0.60403
[1]	validation_0-auc:0.67099	validation_1-auc:0.67000
[2]	validation_0-auc:0.68364	validation_1-auc:0.68244
[3]	validation_0-auc:0.69813	validation_1-auc:0.69717
[4]	validation_0-auc:0.70122	validation_1-auc:0.70025
[5]	validation_0-auc:0.71013	validation_1-auc:0.70937
[6]	validation_0-auc:0.70980	validation_1-auc:0.70893
[7]	validation_0-auc:0.70979	validation_1-auc:0.70877
[8]	validation_0-auc:0.71412	validation_1-auc:0.71309
[9]	validation_0-auc:0.71645	validation_1-auc:0.71545
[10]	validation_0-auc:0.71725	validation_1-auc:0.71616
[11]	validation_0-auc:0.71701	validation_1-auc:0.71597
[12]	validation_0-auc:0.71683	validation_1-auc:0.71584
[13]	validation_0-auc:0.71747	validation_1-auc:0.71653
[14]	validation_0-auc:0.71819	validation_1-auc:0.71721
[15]	validation_0-auc:0.71844	validation_1-auc:0.71740
[16]	validation_0-auc:0.71896	validation_1-auc:0.71791
[17]	validation_0-auc:0.71889	validation_1-auc:0.71779
[18]	validation_0-au

XGBClassifier(alpha=0.1, base_score=None, booster=None, callbacks=None,
              colsample_bylevel=0.5, colsample_bynode=0.5, colsample_bytree=0.5,
              early_stopping_rounds=None, enable_categorical=False, eta=0.01,
              eval_metric='auc', feature_types=None, gamma=0.2, gpu_id=0,
              grow_policy=None, importance_type=None,
              interaction_constraints=None, lambda=1.5, learning_rate=None,
              max_bin=None, max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=5, max_depth=8, max_leaves=None,
              min_child_weight=5, missing=nan, monotone_constraints=None,
              n_estimators=4000, n_jobs=8, ...)

In [9]:
import matplotlib
matplotlib.use('TkAgg')
xgb.plot_importance(model)
feature_important = model.get_booster().get_score(importance_type='weight')
keys = list(feature_important.keys())
values = list(feature_important.values())
argsort_ = np.argsort(values)
for ind in argsort_:
    print('{} {}'.format(keys[ind],values[ind]))

home_ownership_2 1.0
emp_length_and_purpose_139 1.0
emp_length_and_purpose_31 1.0
emp_length_and_purpose_94 1.0
emp_length_and_purpose_17 1.0
emp_length_and_purpose_136 2.0
emp_length_and_purpose_38 3.0
emp_length_and_purpose_3 3.0
emp_length_and_purpose_122 3.0
emp_length_and_purpose_111 10.0
emp_length_and_purpose_143 11.0
emp_length_and_purpose_125 14.0
emp_length_and_purpose_69 15.0
emp_length_and_purpose_61 15.0
emp_length_and_purpose_10 19.0
emp_length_and_purpose_153 24.0
emp_length_and_purpose_78 26.0
emp_length_and_purpose_133 26.0
emp_length_and_purpose_97 26.0
emp_length_and_purpose_137 27.0
emp_length_and_purpose_55 30.0
emp_length_and_purpose_89 30.0
emp_length_and_purpose_59 31.0
emp_length_and_purpose_131 33.0
emp_length_and_purpose_120 33.0
emp_length_and_purpose_91 35.0
emp_length_and_purpose_24 36.0
emp_length_and_purpose_134 37.0
emp_length_and_purpose_92 37.0
emp_length_and_purpose_47 40.0
emp_length_and_purpose_45 41.0
emp_length_and_purpose_96 41.0
emp_length_and_

In [10]:
g = xgb.to_graphviz(model, num_trees=2999)
g.render(filename=os.path.join(conf.data_folder,'xgboost_plot.dot'))
# dot -Tsvg xgboost_plot.dot > o_.svg

'/home/user/FTC/ROW_DATA/xgboost_plot.dot.pdf'

In [11]:
results = model.evals_result()
loss_diff = np.array(results['validation_1'][eval_metric])-np.array(results['validation_0'][eval_metric])
train_loss = np.array(results['validation_1'][eval_metric])
val_loss = np.array(results['validation_0'][eval_metric])
simple_plot(np.arange(0,len(loss_diff)), loss_diff,title=r'$L_{train}-L_{val}$')
simple_plot_y1y2(np.arange(0,len(loss_diff)),train_loss,val_loss, r'$L_{train}$',r'$L_{val}$',title=eval_metric)


# plt.show()

(<Figure size 1600x900 with 1 Axes>, <Axes: title={'center': 'auc'}>)

In [12]:
# preds_ = model.predict(X_validation)
# probas_ = model.predict_proba(X_validation)
# fpr,tpr,tr = metrics.roc_curve(y_validation,probas_[:,1],pos_label=1)
# # simple_plot(fpr,tpr,title='ROC XGBoost')
# print('AUC {}'.format(metrics.auc(fpr,tpr)))
# print('Gini {}'.format(Gini(fpr,tpr)))

In [13]:
X_eval = pd.read_csv(conf.X_test_dataset,index_col=False)

In [14]:
print([el for el in X_eval])

['acc_now_delinq', 'annual_inc', 'chargeoff_within_12_mths', 'collections_12_mths_ex_med', 'delinq_2yrs', 'dti', 'earliest_cr_line', 'funded_amnt', 'inq_last_12m', 'installment', 'int_rate', 'issue_d', 'loan_amnt', 'mort_acc', 'mths_since_last_delinq', 'mths_since_recent_bc_dlq', 'mths_since_recent_inq', 'num_accts_ever_120_pd', 'num_actv_bc_tl', 'num_rev_accts', 'num_sats', 'num_tl_120dpd_2m', 'num_tl_30dpd', 'num_tl_90g_dpd_24m', 'num_tl_op_past_12m', 'open_acc', 'open_il_24m', 'open_rv_24m', 'percent_bc_gt_75', 'pub_rec', 'pub_rec_bankruptcies', 'revol_util', 'tax_liens', 'total_acc', 'zip_code', 'fico_range_mid', 'emp_length_and_purpose_1', 'emp_length_and_purpose_2', 'emp_length_and_purpose_3', 'emp_length_and_purpose_4', 'emp_length_and_purpose_5', 'emp_length_and_purpose_6', 'emp_length_and_purpose_7', 'emp_length_and_purpose_8', 'emp_length_and_purpose_9', 'emp_length_and_purpose_10', 'emp_length_and_purpose_11', 'emp_length_and_purpose_12', 'emp_length_and_purpose_13', 'emp_le

In [15]:
eval_preds_ = model.predict(X_eval)
eval_proba_ = model.predict_proba(X_eval)

In [16]:
probability_of_positive_class = eval_proba_[:,1]
probability_of_negative_class = eval_proba_[:,0]

In [17]:
# fig1,ax1 = plt.subplots(2,2)
# fig1.set_size_inches(16,9)
# ax1[0][0].hist(probability_of_positive_class)
# ax1[0][0].set_xlim([0.0,1.0])
# ax1[0][0].set_title('eval positive '+str(np.median(probability_of_positive_class)))
# ax1[1][0].hist(probability_of_negative_class)
# ax1[1][0].set_title('eval negative '+str(np.median(probability_of_negative_class)))
# ax1[1][0].set_xlim([0.0,1.0])

# validation_posive_distrib = probas_[:,1]
# validation_negative_distrib = probas_[:,0]
# ax1[0][1].hist(validation_posive_distrib)
# ax1[0][1].set_xlim([0.0,1.0])
# ax1[0][1].set_title('train positive '+str(np.median(validation_posive_distrib)))
# ax1[1][1].hist(validation_negative_distrib)
# ax1[1][1].set_title('train negative '+str(np.median(validation_negative_distrib)))
# ax1[1][1].set_xlim([0.0,1.0])




In [18]:
def make_submission(positive_probabilities):
    d = {'index': [i for i in range(0,len(positive_probabilities))], 'loan_status': positive_probabilities}
    o_df = pd.DataFrame(data=d)
    submission_csv = os.path.join('answer.csv')
    submission_zip = os.path.join(conf.data_folder,'answer.zip')
    o_df.to_csv(submission_csv,index=False)
    os.system('rm {}'.format(submission_zip))
    os.system('zip {} {}'.format(submission_zip, submission_csv))

In [19]:
make_submission(positive_probabilities=probability_of_positive_class)

  adding: answer.csv (deflated 61%)


In [20]:
plt.show()